In [6]:
import os
import tweepy 
import pandas as pd
import csv
import string

import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
from textblob import TextBlob
from gensim.models.phrases import Phrases, Phraser
import re


In [7]:
from nltk.stem import WordNetLemmatizer 

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Code to fetch tweets


In [ ]:

consumer_key= 'Zm88MmZ4UYJ1o1SqZEUl5blzl'
consumer_secret= '4u0TPzbqEoWt7xoH4mB0ttZyryPRZtKrSHqqvg4xG3r0PLVsm1'
access_token= '1363507078325747721-TiUTfvsD0TfDXzwO0u8HqG0kcxQAEz'
access_token_secret= 'rSGdxdY1d5LokxQUo4xmzvMGhOV8BslNiZGry9DanLpl3'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Open/Create a file to append data
csvFile = open('farmersprotest.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

q=''
ids = set()
for tweet in tweepy.Cursor(api.search, 
                    q="#FarmersProtest", 
                    Since="2021-01-01", 
                    tweet_mode='extended',
                    #until="2014-02-15", 
                    lang="en").items(1000):
    if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
     #Write a row to the csv file/ I use encode utf-8
        csvWriter.writerow([tweet.created_at, tweet.full_text, tweet.favorite_count, tweet.retweet_count, tweet.id, tweet.user.screen_name])
     #print "...%s tweets downloaded so far" % (len(tweet.id))
        ids.add(tweet.id) # add new id
        print ("number of unique ids seen so far: {}",format(len(ids)))
csvFile.close()

In [8]:
df=pd.read_csv('C://Users//Athira Shankar//Downloads//farmersprotest_final.csv',header=None,names=['created_at'\
                                                                                             ,'text'\
                                                                                             ,'favorite_count'\
                                                                                            ,'retweet_count'\
                                                                                            ,'id'\
                                                                                            ,'screen_name'])

In [9]:
df

,created_at,text,favorite_count,retweet_count,id,screen_name
0,05-04-2021 04:38,"whenever elections happens in country, nationa...",0,0,1.378930e+18,singh_harry07
1,05-04-2021 04:37,Martyred Or Killed?\nkilled for Dirty politics...,1,2,1.378930e+18,Navjotk17902569
2,05-04-2021 04:37,@ChrisCuomo The Indian media is pro-Modi and w...,0,0,1.378930e+18,MalikAvni
3,05-04-2021 04:36,"Yes please raise your voice, we need you to sp...",0,0,1.378930e+18,Aman_Sandhu05
4,05-04-2021 04:36,@sukhisukhii @Karanveer_135 @AbhinayDubey @Twe...,2,0,1.378930e+18,89MaNdEePp
...,...,...,...,...,...,...
11136,30-03-2021 08:15,No Farmer's No Food\n#sikh\n#Sikhism\n#Khalsa\...,1,2,1.376810e+18,manjits47241265
11137,30-03-2021 08:15,@timesofindia They refused to host the tricolo...,0,0,1.376810e+18,1Kuldeep2
11138,30-03-2021 08:14,@timesofindia They refused to host the tricolo...,0,0,1.376810e+18,1Kuldeep2
11139,30-03-2021 08:14,News: Farmer leaders giving speech at Gazipur ...,10,3,1.376810e+18,KrishakNews


# Data Cleaning

In [10]:
#convert the text to lowercase
df['preprocessed']=df['text'].str.lower().str.replace('#farmersprotest'," ")

In [11]:
#function to remove a given pattern
def remove_pattern(input_text,pattern):
    p=re.findall(pattern,input_text)
    for i in p:
        input_text=re.sub(i,"",input_text)
    return input_text

In [12]:
df['preprocessed']=df['preprocessed'].apply(lambda x: remove_pattern(x,"@[\w]*"))
#Removing Usernames
# clean_text=remove_pattern(remove_spaces,"@[\w]*")

In [13]:
#removes punctuations,hyperlinks
df['preprocessed']=df['preprocessed'].apply(lambda x: " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", x).split()))


In [15]:
#Removing white spaces
df['preprocessed']=df['preprocessed'].apply(lambda x: re.sub('/s+'," ",x))

In [16]:
#Function for stopword removal
def removeStopWords(x):
    words=nltk.tokenize.word_tokenize(x)
    words_new=[i for i in words if i not in stop]
    return words_new
    
    

In [17]:
df['token_words']=df['preprocessed'].apply(removeStopWords)


In [18]:
#Lemmatization
df['lemmatized_words']=df['token_words'].apply(lambda x:[ lemmatizer.lemmatize(text) for text in x])

In [19]:
df['lemmatized_sentence']=df['lemmatized_words'].apply(lambda x:' '.join(x))

In [20]:
df['lemmatized_sentence']=df['lemmatized_sentence'].str.replace('shaheedjawan',"")

In [21]:
df['lemmatized_sentence']=df['lemmatized_sentence'].str.replace('shaheedkisan',"")

In [22]:
df['lemmatized_sentence']=df['lemmatized_sentence'].str.replace('shaheed',"")

# Method 1 - TextBlob library

In [23]:
# Obtaining the polarity of each sentence using TextBlob
df['polarity']=df['lemmatized_sentence'].apply(lambda x:TextBlob(x).sentiment.polarity)

In [24]:
#Function for sentiment classification
def sentiment_classification(x):
    if(x<0):
        return 'Negative'
    elif(x>0):
        return 'Positive'
    else:
        return 'Neutral'

In [25]:
df['sentiment_class']=df['polarity'].apply(sentiment_classification)

In [26]:
df['sentiment_class'].value_counts(normalize=True)

Neutral     0.433713
Positive    0.397451
Negative    0.168836
Name: sentiment_class, dtype: float64

In [36]:
df

,created_at,text,favorite_count,retweet_count,id,screen_name,preprocessed,token_words,lemmatized_words,lemmatized_sentence,polarity,sentiment_class
0,05-04-2021 04:38,"whenever elections happens in country, nationa...",0,0,1.378930e+18,singh_harry07,whenever elections happens in country national...,"[whenever, elections, happens, country, nation...","[whenever, election, happens, country, nationa...",whenever election happens country national sec...,0.000000,Neutral
1,05-04-2021 04:37,Martyred Or Killed?\nkilled for Dirty politics...,1,2,1.378930e+18,Navjotk17902569,martyred or killed killed for dirty politics s...,"[martyred, killed, killed, dirty, politics, sh...","[martyred, killed, killed, dirty, politics, sh...",martyred killed killed dirty politics shame bj...,-0.280000,Negative
2,05-04-2021 04:37,@ChrisCuomo The Indian media is pro-Modi and w...,0,0,1.378930e+18,MalikAvni,the indian media is pro modi and won t cover t...,"[indian, media, pro, modi, cover, farmers, sid...","[indian, medium, pro, modi, cover, farmer, sid...",indian medium pro modi cover farmer side story...,0.142857,Positive
3,05-04-2021 04:36,"Yes please raise your voice, we need you to sp...",0,0,1.378930e+18,Aman_Sandhu05,yes please raise your voice we need you to spe...,"[yes, please, raise, voice, need, speak, right...","[yes, please, raise, voice, need, speak, right...",yes please raise voice need speak right,0.285714,Positive
4,05-04-2021 04:36,@sukhisukhii @Karanveer_135 @AbhinayDubey @Twe...,2,0,1.378930e+18,89MaNdEePp,whom are you guys dbating with not a single tw...,"[guys, dbating, single, tweet, mind, using, ot...","[guy, dbating, single, tweet, mind, using, oth...",guy dbating single tweet mind using others twe...,-0.071429,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...
11136,30-03-2021 08:15,No Farmer's No Food\n#sikh\n#Sikhism\n#Khalsa\...,1,2,1.376810e+18,manjits47241265,no farmer s no food sikh sikhism khalsa stopha...,"[farmer, food, sikh, sikhism, khalsa, stophate...","[farmer, food, sikh, sikhism, khalsa, stophate...",farmer food sikh sikhism khalsa stophateagains...,0.000000,Neutral
11137,30-03-2021 08:15,@timesofindia They refused to host the tricolo...,0,0,1.376810e+18,1Kuldeep2,they refused to host the tricolour until 2002 ...,"[refused, host, tricolour, 2002, stood, indepe...","[refused, host, tricolour, 2002, stood, indepe...",refused host tricolour 2002 stood independence...,-0.250000,Negative
11138,30-03-2021 08:14,@timesofindia They refused to host the tricolo...,0,0,1.376810e+18,1Kuldeep2,they refused to host the tricolour until 2002 ...,"[refused, host, tricolour, 2002, stood, indepe...","[refused, host, tricolour, 2002, stood, indepe...",refused host tricolour 2002 stood independence...,-0.250000,Negative
11139,30-03-2021 08:14,News: Farmer leaders giving speech at Gazipur ...,10,3,1.376810e+18,KrishakNews,news farmer leaders giving speech at gazipur b...,"[news, farmer, leaders, giving, speech, gazipu...","[news, farmer, leader, giving, speech, gazipur...",news farmer leader giving speech gazipur borde...,0.000000,Neutral


In [54]:
sent = [row for row in df.lemmatized_words]

# Method 2- Wordnet and Kmeans

In [48]:
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

In [49]:
phrases = Phrases(sent, min_count=1, progress_per=50000)

In [55]:
bigram = Phraser(phrases)

In [56]:
sentences = bigram[sent]


In [57]:
import multiprocessing
from time import time 


In [58]:
#Setting the word2vec model
w2v_model = Word2Vec(min_count=2,
                     seed=1,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

Time to build vocab: 0.02 mins


In [59]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_model.epochs)

(564964, 674215)

In [137]:
word_vectors = Word2Vec.load("word2vec.model").wv


In [60]:
word_vectors=w2v_model.wv

In [61]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))


In [62]:
len(word_vectors.index_to_key)

6264

In [65]:
word_vectors.most_similar("horrible")

[('high', 0.9882523417472839),
 ('begun', 0.9867973923683167),
 ('goon', 0.9865074157714844),
 ('watch', 0.986379861831665),
 ('terrorist', 0.985410213470459),
 ('killed', 0.9847357869148254),
 ('violence', 0.9844675660133362),
 ('terror', 0.9841641187667847),
 ('fascism', 0.9830507636070251),
 ('ambani', 0.9818781018257141)]

In [68]:
word_vectors.most_similar("farmer")

[('protesting', 0.9812927842140198),
 ('peacefully', 0.9800936579704285),
 ('protestors', 0.9761881232261658),
 ('harassing', 0.9717051982879639),
 ('wrapped', 0.9711225032806396),
 ('due', 0.9695504903793335),
 ('fancy', 0.9684891104698181),
 ('seeking', 0.9682461619377136),
 ('rollback', 0.9681868553161621),
 ('justified', 0.9644160270690918)]

In [78]:
word_vectors.similar_by_vector(model.cluster_centers_[0], topn=300, restrict_vocab=None)

[('used', 0.9994137287139893),
 ('mean', 0.9993039965629578),
 ('left', 0.999300479888916),
 ('order', 0.9992573857307434),
 ('video', 0.9992283582687378),
 ('visit', 0.999226450920105),
 ('money', 0.9991975426673889),
 ('art', 0.9991844296455383),
 ('na', 0.9991757869720459),
 ('etc', 0.9991629719734192),
 ('president', 0.9991469383239746),
 ('loss', 0.999098539352417),
 ('chautala', 0.9990964531898499),
 ('already', 0.9990944266319275),
 ('another', 0.999091386795044),
 ('full', 0.9990886449813843),
 ('punjabi', 0.9990456104278564),
 ('hand', 0.9989734888076782),
 ('game', 0.9989645481109619),
 ('population', 0.9989399909973145),
 ('force', 0.9989343285560608),
 ('onthisday', 0.9989259243011475),
 ('around', 0.9989204406738281),
 ('leadership', 0.9989070296287537),
 ('piece', 0.9988948702812195),
 ('rice', 0.9988939762115479),
 ('also', 0.9988901615142822),
 ('nhi', 0.9988798499107361),
 ('society', 0.9988783001899719),
 ('zee', 0.9988723397254944),
 ('deepsidhu', 0.998863697052002),

## Assigning clusters

In [79]:
positive_cluster_index = 0
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]


In [80]:
word_vectors.key_to_index.keys()

dict_keys(['farmer', 'r', 'bjp', 'e', 'lashkar', 'farmersagainstatrocities', 'amp', 'india', 'modi', 'nationaljumladay', 'people', 'law', 'farm', 'support', 'right', 'bill', 'nation', 'stop', 'govt', 'indian', 'u', 'stophateagainstfarmers', 'protest', 'one', 'please', 'leader', 'day', 'government', 'new', 'good', 'history', 'tweet', 'today', '3', 'country', 'shaheedkisan', 'shaheedjawan', 'like', '2', 'attack', 'repeal', 'back', 'need', 'let', 'life', 'keep', 'hate', 'create', 'still', 'time', 'world', 'get', 'stand', 'shah', 'make', 'save', 'april', '4', 'say', 'know', 'overturning', 'protesting', 'take', 'sikh', 'always', '1', 'fight', 'humanrights', 'vote', 'give', 'hello', 'jumla', 'want', 'voice', 'police', 'never', 'hashtag', 'msp', 'go', 'food', 'democracy', 'rollback', 'must', 'delhi', 'win', 'going', 'ji', 'listen', 'evm', 'black', 'call', 'put', 'see', 'power', 'justified', 'ur', 'atrocity', 'gobackmodi', 'seeking', 'assam', 'fighting', 'made', 'every', 'respect', 'better', '

In [81]:
words = pd.DataFrame(word_vectors.key_to_index.keys())


In [82]:
words

,0
0,farmer
1,r
2,bjp
3,e
4,lashkar
...,...
6259,ass
6260,bull
6261,flipkart
6262,globalfenkuday


In [83]:
import numpy as np

In [84]:
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [85]:
words

,words,vectors,cluster
0,farmer,"[0.1367595, 0.48981902, -0.03982155, -0.410703...",1
1,r,"[0.48500717, 1.6384904, -0.47608593, 0.8438523...",1
2,bjp,"[0.4315125, 1.138526, -0.47621134, 1.0055408, ...",1
3,e,"[0.28082857, 1.5197388, -0.45454666, 0.4386389...",1
4,lashkar,"[0.31815362, 1.4695939, -0.57647055, 0.5631407...",1
...,...,...,...
6259,ass,"[0.01816884, 0.036974225, -0.011086602, 0.0165...",0
6260,bull,"[0.026098665, 0.013915288, -0.022585183, 0.004...",0
6261,flipkart,"[0.05644551, 0.046611216, -0.029664366, -0.002...",0
6262,globalfenkuday,"[-0.0019230692, 0.002727638, -0.0034496454, -0...",0


In [87]:
#Assigning 1 if in positive cluster else -1
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]


In [88]:
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)


In [89]:
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [90]:
words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)

In [91]:
#Creating a sentiment dictionary
sentiment_dict = dict(zip(words['words'], words['sentiment_coeff']))

# TFIDF

In [92]:

from sklearn.feature_extraction.text import TfidfVectorizer


In [93]:
tfidf = TfidfVectorizer(tokenizer=lambda y: y.split(), norm=None)


In [94]:
tfidf

TfidfVectorizer(norm=None, tokenizer=<function <lambda> at 0x0000026115628820>)

In [97]:
tfidf.fit(df.lemmatized_sentence)

TfidfVectorizer(norm=None, tokenizer=<function <lambda> at 0x0000026115628820>)

In [98]:
features = pd.Series(tfidf.get_feature_names())

In [99]:
transformed = tfidf.transform(df.lemmatized_sentence)

In [100]:
transformed

<11141x12194 sparse matrix of type '<class 'numpy.float64'>'
	with 126074 stored elements in Compressed Sparse Row format>

In [103]:
def create_tfidf_dictionary(x, transformed_file, features):
 
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x.lemmatized_sentence.split()))

In [105]:
%%time
replaced_tfidf_scores =df.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1)
#this step takes around 3-4 minutes minutes to calculate

Wall time: 6.57 s


In [106]:
replaced_tfidf_scores

0        [7.679419700965502, 5.3912233454235565, 7.4281...
1        [7.610426829478551, 19.159954193569305, 19.159...
2        [3.9434522107526644, 5.301197193765836, 6.9511...
3        [5.69350421729649, 4.282995598056004, 5.864129...
4        [6.069981788531402, 9.625329850020815, 6.14408...
                               ...                        
11136    [2.069424756409469, 4.9906008617911795, 4.9906...
11137    [6.654915384451114, 6.654915384451114, 6.70755...
11138    [6.654915384451114, 6.654915384451114, 6.70755...
11139    [5.818667360250496, 2.069424756409469, 4.22040...
11140    [6.821969469114281, 8.239035488900925, 8.37256...
Length: 11141, dtype: object

In [107]:
sentiment_dict

{'farmer': -0.307809996075739,
 'r': -0.20202941229847732,
 'bjp': -0.2411646716302142,
 'e': -0.2339880731470427,
 'lashkar': -0.21957632198472557,
 'farmersagainstatrocities': -0.7594865822591885,
 'amp': -0.42219479951134653,
 'india': -0.3353517172310396,
 'modi': -0.22761841533831448,
 'nationaljumladay': -0.6041213479538147,
 'people': -0.3556537023121394,
 'law': -0.1604864338034591,
 'farm': -0.14604203642575841,
 'support': -0.2818580849734844,
 'right': -0.30232892783572235,
 'bill': -0.14486224187320043,
 'nation': -0.20074430599052612,
 'stop': -0.2561524016991294,
 'govt': -0.24820593628784177,
 'indian': -0.23555381103215456,
 'u': -0.3131579863207159,
 'stophateagainstfarmers': -0.3551320074824742,
 'protest': -0.44066730362085754,
 'one': -0.8070037854184893,
 'please': -0.15689593143071925,
 'leader': -0.22422678148711067,
 'day': -0.45886095734424265,
 'government': -0.5404444632081149,
 'new': -0.21022405152394685,
 'good': -0.2511801152925348,
 'history': -0.1938564

In [108]:
#replacing tfidf score with word2vec sentiment score
def replace_sentiment_words(word, sentiment_dict):
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out

In [109]:
replaced_closeness_scores = df.lemmatized_sentence.apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))



In [110]:
replaced_closeness_scores

0        [5.996491165273378, -0.8849977836176851, 0.922...
1        [1.2776218441094094, -0.8260927341040494, -0.8...
2        [-0.23555381103215456, -1.159392718795086, 0.9...
3        [-1.6770981492698858, -0.15689593143071925, -1...
4        [-1.3170520399406918, 0, -1.145386061496251, -...
                               ...                        
11136    [-0.307809996075739, -0.6658766789724803, -0.4...
11137    [1.1911306944899902, 0.8473556203987233, 0.678...
11138    [1.1911306944899902, 0.8473556203987233, 0.678...
11139    [-1.4570738945827149, -0.307809996075739, -0.2...
11140    [-1.3778447272849255, 8.514892389564196, 8.470...
Name: lemmatized_sentence, Length: 11141, dtype: object

In [111]:
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, df.lemmatized_sentence, df.text]).T
replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'lemmatized_sentence', 'tweets']


In [112]:
replacement_df

,sentiment_coeff,tfidf_scores,lemmatized_sentence,tweets
0,"[5.996491165273378, -0.8849977836176851, 0.922...","[7.679419700965502, 5.3912233454235565, 7.4281...",whenever election happens country national sec...,"whenever elections happens in country, nationa..."
1,"[1.2776218441094094, -0.8260927341040494, -0.8...","[7.610426829478551, 19.159954193569305, 19.159...",martyred killed killed dirty politics shame bj...,Martyred Or Killed?\nkilled for Dirty politics...
2,"[-0.23555381103215456, -1.159392718795086, 0.9...","[3.9434522107526644, 5.301197193765836, 6.9511...",indian medium pro modi cover farmer side story...,@ChrisCuomo The Indian media is pro-Modi and w...
3,"[-1.6770981492698858, -0.15689593143071925, -1...","[5.69350421729649, 4.282995598056004, 5.864129...",yes please raise voice need speak right,"Yes please raise your voice, we need you to sp..."
4,"[-1.3170520399406918, 0, -1.145386061496251, -...","[6.069981788531402, 9.625329850020815, 6.14408...",guy dbating single tweet mind using others twe...,@sukhisukhii @Karanveer_135 @AbhinayDubey @Twe...
...,...,...,...,...
11136,"[-0.307809996075739, -0.6658766789724803, -0.4...","[2.069424756409469, 4.9906008617911795, 4.9906...",farmer food sikh sikhism khalsa stophateagains...,No Farmer's No Food\n#sikh\n#Sikhism\n#Khalsa\...
11137,"[1.1911306944899902, 0.8473556203987233, 0.678...","[6.654915384451114, 6.654915384451114, 6.70755...",refused host tricolour 2002 stood independence...,@timesofindia They refused to host the tricolo...
11138,"[1.1911306944899902, 0.8473556203987233, 0.678...","[6.654915384451114, 6.654915384451114, 6.70755...",refused host tricolour 2002 stood independence...,@timesofindia They refused to host the tricolo...
11139,"[-1.4570738945827149, -0.307809996075739, -0.2...","[5.818667360250496, 2.069424756409469, 4.22040...",news farmer leader giving speech gazipur borde...,News: Farmer leaders giving speech at Gazipur ...


## Matrix multiplication 

In [113]:
replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)


##  converting bool (boolean to integer )

In [114]:
replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')
# replacement_df['sentiment'] = [1 if i==1 else 0 for i in replacement_df.sentiment]

In [117]:
replacement_df['prediction'].value_counts(normalize=True)

1    0.598779
0    0.401221
Name: prediction, dtype: float64

In [116]:
replacement_df

,sentiment_coeff,tfidf_scores,lemmatized_sentence,tweets,sentiment_rate,prediction
0,"[5.996491165273378, -0.8849977836176851, 0.922...","[7.679419700965502, 5.3912233454235565, 7.4281...",whenever election happens country national sec...,"whenever elections happens in country, nationa...",40.100976,1
1,"[1.2776218441094094, -0.8260927341040494, -0.8...","[7.610426829478551, 19.159954193569305, 19.159...",martyred killed killed dirty politics shame bj...,Martyred Or Killed?\nkilled for Dirty politics...,49.939638,1
2,"[-0.23555381103215456, -1.159392718795086, 0.9...","[3.9434522107526644, 5.301197193765836, 6.9511...",indian medium pro modi cover farmer side story...,@ChrisCuomo The Indian media is pro-Modi and w...,46.759717,1
3,"[-1.6770981492698858, -0.15689593143071925, -1...","[5.69350421729649, 4.282995598056004, 5.864129...",yes please raise voice need speak right,"Yes please raise your voice, we need you to sp...",-31.407065,0
4,"[-1.3170520399406918, 0, -1.145386061496251, -...","[6.069981788531402, 9.625329850020815, 6.14408...",guy dbating single tweet mind using others twe...,@sukhisukhii @Karanveer_135 @AbhinayDubey @Twe...,89.836011,1
...,...,...,...,...,...,...
11136,"[-0.307809996075739, -0.6658766789724803, -0.4...","[2.069424756409469, 4.9906008617911795, 4.9906...",farmer food sikh sikhism khalsa stophateagains...,No Farmer's No Food\n#sikh\n#Sikhism\n#Khalsa\...,15.178250,1
11137,"[1.1911306944899902, 0.8473556203987233, 0.678...","[6.654915384451114, 6.654915384451114, 6.70755...",refused host tricolour 2002 stood independence...,@timesofindia They refused to host the tricolo...,-32.226392,0
11138,"[1.1911306944899902, 0.8473556203987233, 0.678...","[6.654915384451114, 6.654915384451114, 6.70755...",refused host tricolour 2002 stood independence...,@timesofindia They refused to host the tricolo...,-32.226392,0
11139,"[-1.4570738945827149, -0.307809996075739, -0.2...","[5.818667360250496, 2.069424756409469, 4.22040...",news farmer leader giving speech gazipur borde...,News: Farmer leaders giving speech at Gazipur ...,12.937672,1


In [204]:
df

,created_at,text,favorite_count,retweet_count,id,screen_name,preprocessed,token_words,lemmatized_words,lemmatized_sentence,polarity,sentiment_class
0,05-04-2021 04:38,"whenever elections happens in country, nationa...",0.0,0.0,1.378930e+18,singh_harry07,whenever elections happens in country national...,"[whenever, elections, happens, country, nation...","[whenever, election, happens, country, nationa...",whenever election happens country national sec...,0.000000,Neutral
1,05-04-2021 04:37,Martyred Or Killed?\nkilled for Dirty politics...,1.0,2.0,1.378930e+18,Navjotk17902569,martyred or killed killed for dirty politics s...,"[martyred, killed, killed, dirty, politics, sh...","[martyred, killed, killed, dirty, politics, sh...",martyred killed killed dirty politics shame bj...,-0.280000,Negative
2,05-04-2021 04:37,@ChrisCuomo The Indian media is pro-Modi and w...,0.0,0.0,1.378930e+18,MalikAvni,the indian media is pro modi and won t cover t...,"[indian, media, pro, modi, cover, farmers, sid...","[indian, medium, pro, modi, cover, farmer, sid...",indian medium pro modi cover farmer side story...,0.142857,Positive
3,05-04-2021 04:36,"Yes please raise your voice, we need you to sp...",0.0,0.0,1.378930e+18,Aman_Sandhu05,yes please raise your voice we need you to spe...,"[yes, please, raise, voice, need, speak, right...","[yes, please, raise, voice, need, speak, right...",yes please raise voice need speak right,0.285714,Positive
4,05-04-2021 04:36,@sukhisukhii @Karanveer_135 @AbhinayDubey @Twe...,2.0,0.0,1.378930e+18,89MaNdEePp,whom are you guys dbating with not a single tw...,"[guys, dbating, single, tweet, mind, using, ot...","[guy, dbating, single, tweet, mind, using, oth...",guy dbating single tweet mind using others twe...,-0.071429,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...
11138,30-03-2021 08:14,@timesofindia They refused to host the tricolo...,0.0,0.0,1.376810e+18,1Kuldeep2,they refused to host the tricolour until 2002 ...,"[refused, host, tricolour, 2002, stood, indepe...","[refused, host, tricolour, 2002, stood, indepe...",refused host tricolour 2002 stood independence...,-0.250000,Negative
11139,30-03-2021 08:14,News: Farmer leaders giving speech at Gazipur ...,10.0,3.0,1.376810e+18,KrishakNews,news farmer leaders giving speech at gazipur b...,"[news, farmer, leaders, giving, speech, gazipu...","[news, farmer, leader, giving, speech, gazipur...",news farmer leader giving speech gazipur borde...,0.000000,Neutral
11140,30-03-2021 08:14,Was Looking At These Hashtags #StopAsianHate #...,1.0,3.0,1.376810e+18,ErFarm_,was looking at these hashtags stopasianhate st...,"[looking, hashtags, stopasianhate, stopasianha...","[looking, hashtags, stopasianhate, stopasianha...",looking hashtags stopasianhate stopasianhatecr...,0.357143,Positive
11141,NaN,I'm a very good guy who does good things,NaN,NaN,NaN,Test1,i m a very good guy who does good things,"[good, guy, good, things]","[good, guy, good, thing]",good guy good thing,0.700000,Positive
